In [1]:
import numpy as np
import torch

from ssd.utils_ssd300 import dboxes300_coco
from ssd.create_model import nvidia_ssd
from ssd.utils_ssd300 import Encoder
from ssd.model import Loss, SSD300
from ssd.SSD_Transformers import SSDTransformer
from utils.utils import set_seed
from train_model import train_model
from ssd.model_eval import model_evaluate


from torch.utils.data import DataLoader

from config.config import Configs
from ssd.model_eval import model_evaluate

from ssd.decode_results import Processing as processing
from ssd.loaser_crowdhuman import CrowdHuman


c:\Users\123\Desktop\torch cuda\.venv\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
c:\Users\123\Desktop\torch cuda\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda'
ssd_300 = nvidia_ssd(device=device, pretrainded_custom=False, pretrained_default=True)

feat_ext_w = ssd_300.feature_extractor.state_dict()

model = SSD300(label_num=3)
model = model.cuda()

model.feature_extractor.load_state_dict(feat_ext_w)


<All keys matched successfully>

In [5]:
batch_size = 32
device = "cuda"

for param in model.parameters():
    param.requires_grad = True

for param in model.feature_extractor.parameters():
    param.requires_grad = False

dboxes = dboxes300_coco()
encoder = Encoder(dboxes)

loss_func = Loss(dboxes, device=device)
transformers_ssd_train = SSDTransformer(dboxes, size=(300, 300), val=False)
transformers_ssd_val = SSDTransformer(dboxes, size=(300, 300), val=True)
set_seed()
data_train = CrowdHuman(
    img_folder="data\\train\\",
    annotate_file="data\\annotation_train.json",
    transform=transformers_ssd_train,
    use_head_bbx=True,
)


train_dataloader = DataLoader(
    data_train,
    batch_size=batch_size,
    shuffle=True 
)


data_val = CrowdHuman(
    img_folder="data\\val\\",
    annotate_file="data\\annotation_val.json",
    transform=transformers_ssd_val,
    use_head_bbx=True,
)

val_dataloader = DataLoader(
    data_val,
    batch_size=batch_size,
    shuffle=False,  
)


# https://www.kaggle.com/code/billiemage/understand-lr-scheduler-with-simple-examples
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=3.25e-3, momentum=0.9, nesterov=True)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, patience=4,threshold = 1e-3, verbose=True, factor=0.5, mode = 'min'
)

load annotation file:  data\annotation_train.json
load annotation file:  data\annotation_val.json


In [6]:
train_model(
    model=model,
    encoder = encoder,
    num_epoch=80,
    optimizer=optimizer,
    loss_func = loss_func,
    train_dataloader = train_dataloader,
    val_dataloader = val_dataloader,
    device  = device,
    scheduler=scheduler,
    path_to_save=Configs.path_to_save_state_model,
    use_pick_best=Configs.use_pick_best_in_eval
)


EPOCH 1 of 80
Training


 26%|██▌       | 61/235 [05:31<15:46,  5.44s/it]


KeyboardInterrupt: 

In [8]:
x = model_evaluate(
    model = model,
    encoder = encoder,
    val_dataloader = val_dataloader,
    device=device,
    use_pick_best=False,
    threshold = Configs.decode_result["pic_threshold"]
)

Evaluating


100%|██████████| 47/47 [18:14<00:00, 23.29s/it]
c:\Users\123\Desktop\torch cuda\.venv\lib\site-packages\torchmetrics\utilities\prints.py:42: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)  # noqa: B028


{'map': tensor(0.1424),
 'map_50': tensor(0.3463),
 'map_75': tensor(0.0972),
 'map_small': tensor(0.0155),
 'map_medium': tensor(0.1636),
 'map_large': tensor(0.3422),
 'mar_1': tensor(0.0230),
 'mar_10': tensor(0.1355),
 'mar_100': tensor(0.2137),
 'mar_small': tensor(0.0521),
 'mar_medium': tensor(0.2502),
 'mar_large': tensor(0.4414),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.),
 'classes': tensor([1, 2], dtype=torch.int32)}